In [8]:
# Paths
import os
import time
import pandas as pd
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = ('ignore::UserWarning,ignore::ConvergenceWarning,ignore::RuntimeWarning')

os.chdir('../..')
from pcmf import pcmf_full, path_plot, plot_ordercolor, plot_cluster_assignments, PCMF_predict_clusters
from p3ca import cluster_metrics, calculate_scores_nonpath
import numpy as np
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering, KMeans, AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import confusion_matrix, accuracy_score
from scipy.optimize import linprog, linear_sum_assignment as linear_assignment

def confusion_matrix_ordered(pred, true):
    def _make_cost_m(cm):
        s = np.max(cm)
        return (- cm + s)
    conf_mat = confusion_matrix(pred,true)
    indexes = linear_assignment(_make_cost_m(conf_mat))
    js = [e for e in sorted(indexes, key=lambda x: x[0])[1]]
    conf_mat_ord = conf_mat[:, js]
    return conf_mat_ord

# Synthetic data
def load_syntheticDataConsensus(n=100000, n_test=10000, p=1000, m=25000, m_test=2500, num_clusters=4, means=[-1.0, 1.0, -0.4, 0.4], r=1, sigma=0.075, density=0.5, plot=False, randomize=False, pcmf_dir="/athena/listonlab/store/amb2022/PCMF/"):
    import numpy as np
    import os
    os.chdir(pcmf_dir)
    from pcmf import generate_cluster_PMD_data
    print('Loading synthetic data, run: '+str(r))
    seeds = [r,r+1,r+2,r+3]
    
    # Cluster sizes
    ms = [m+m_test,m+m_test,m+m_test,m+m_test]
    
    # Get clustered CCA data
    X_clusters, u_true, v_true, _ = generate_cluster_PMD_data(ms, p, sigma, density, num_clusters, means=means) 
    # Training set
    X_c_train = []
    X_in_test = []
    for nc in range(num_clusters):
        X_c_train.append(X_clusters[nc][0:m,:])
        X_in_test.append(X_clusters[nc][m:m+m_test,:])
    X_c_train = np.vstack(X_c_train)
    true_clusters = np.repeat([0,1,2,3],m)
    # Test set
    X_in_test = np.vstack(X_in_test)
    true_clusters_in_test = np.repeat([0,1,2,3],m_test)
    
    # Shape of dataset
    print('X_c_train: ' + str(X_c_train.shape))
    print('Y_train: ' + str(true_clusters.shape))
    print('X_c_test:  '  + str(X_in_test.shape))
    print('Y_test:  '  + str(true_clusters_in_test.shape))
    
    # Randomize training setorder
    if randomize is True:
        print('Randomizing order seed 1234')
        idx_perm = np.random.RandomState(seed=1234).permutation(X_c_train.shape[0])
        X_in = X_c_train[idx_perm,:]
        true_clusters_in = true_clusters[idx_perm]
    else:
        X_in = X_c_train
        true_clusters_in = true_clusters
    
    return X_in, true_clusters_in, X_in_test, true_clusters_in_test, num_clusters, u_true, v_true


In [10]:
X_in, true_clusters_in, X_in_test, true_clusters_in_test, num_clusters, u_true, v_true = load_syntheticDataConsensus(n=100000, n_test=10000, p=10, m=25000, m_test=2500, plot=False, randomize=False)



Loading synthetic data, run: 1
X_c_train: (100000, 10)
Y_train: (100000,)
X_c_test:  (10000, 10)
Y_test:  (10000,)


In [11]:
import pandas as pd
def load_data(X_in,true_clusters_in):
    from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
    scalerX = StandardScaler()
    scalerX.fit(X_in)
    x = scalerX.transform(X_in).astype(np.float32)

    y = pd.factorize(true_clusters_in)[0].astype(np.int32)
    print(y.shape)

    print('samples', x.shape, y.shape)
    return x, y

from torch.utils.data import Dataset
class SyntheticDataset(Dataset):

    def __init__(self):
        self.x, self.y = load_data(X_in,true_clusters_in)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return torch.from_numpy(np.array(self.x[idx])), torch.from_numpy(
            np.array(self.y[idx])), torch.from_numpy(np.array(idx))


In [12]:

#######################################################
# Evaluate Critiron
#######################################################
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI_score, adjusted_rand_score as ARI_score, rand_score as rand_score

def cluster_acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
#     from sklearn.utils.linear_assignment_ import linear_assignment
    from scipy.optimize import linear_sum_assignment as linear_assignment
    ind = linear_assignment(w.max() - w)
#     return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size
    return sum([w[i, j] for i, j in zip(ind[0],ind[1])])*1.0/y_pred.size

In [14]:
# -*- coding: utf-8 -*-
#
# Copyright © dawnranger.
#
# 2018-05-08 10:15 <dawnranger123@gmail.com>
#
# Distributed under terms of the MIT license.
# https://github.com/dawnranger/IDEC-pytorch
from __future__ import print_function, division
import argparse
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from sklearn.metrics import adjusted_rand_score as ari_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.nn import Linear


class AE(nn.Module):

    def __init__(self, n_enc_1, n_enc_2, n_enc_3, n_dec_1, n_dec_2, n_dec_3,
                 n_input, n_z):
        super(AE, self).__init__()

        # encoder
        self.enc_1 = Linear(n_input, n_enc_1)
        self.enc_2 = Linear(n_enc_1, n_enc_2)
        self.enc_3 = Linear(n_enc_2, n_enc_3)

        self.z_layer = Linear(n_enc_3, n_z)

        # decoder
        self.dec_1 = Linear(n_z, n_dec_1)
        self.dec_2 = Linear(n_dec_1, n_dec_2)
        self.dec_3 = Linear(n_dec_2, n_dec_3)

        self.x_bar_layer = Linear(n_dec_3, n_input)

    def forward(self, x):

        # encoder
        enc_h1 = F.relu(self.enc_1(x))
        enc_h2 = F.relu(self.enc_2(enc_h1))
        enc_h3 = F.relu(self.enc_3(enc_h2))

        z = self.z_layer(enc_h3)

        # decoder
        dec_h1 = F.relu(self.dec_1(z))
        dec_h2 = F.relu(self.dec_2(dec_h1))
        dec_h3 = F.relu(self.dec_3(dec_h2))
        x_bar = self.x_bar_layer(dec_h3)

        return x_bar, z


class IDEC(nn.Module):

    def __init__(self,
                 n_enc_1,
                 n_enc_2,
                 n_enc_3,
                 n_dec_1,
                 n_dec_2,
                 n_dec_3,
                 n_input,
                 n_z,
                 n_clusters,
                 alpha=1,
                 pretrain_path='data/ae_mnist.pkl'):
        super(IDEC, self).__init__()
        self.alpha = 1.0
        self.pretrain_path = pretrain_path

        self.ae = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        # cluster layer
        self.cluster_layer = Parameter(torch.Tensor(n_clusters, n_z))
        torch.nn.init.xavier_normal_(self.cluster_layer.data)

    def pretrain(self, path='',pretrain_epochs=200):
        if path == '':
            pretrain_ae(self.ae,pretrain_epochs)
        # load pretrain weights
        self.ae.load_state_dict(torch.load(self.pretrain_path))
        print('load pretrained ae from', path)

    def forward(self, x):

        x_bar, z = self.ae(x)
        # cluster
        q = 1.0 / (1.0 + torch.sum(
            torch.pow(z.unsqueeze(1) - self.cluster_layer, 2), 2) / self.alpha)
        q = q.pow((self.alpha + 1.0) / 2.0)
        q = (q.t() / torch.sum(q, 1)).t()
        return x_bar, q


def target_distribution(q):
    weight = q**2 / q.sum(0)
    return (weight.t() / weight.sum(1)).t()


def pretrain_ae(model,pretrain_epochs=200):
    '''
    pretrain autoencoder
    '''
    train_loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
    print(model)
    optimizer = Adam(model.parameters(), lr=args.lr)
    for epoch in range(pretrain_epochs):
        total_loss = 0.
        for batch_idx, (x, _, _) in enumerate(train_loader):
            x = x.to(device)

            optimizer.zero_grad()
            x_bar, z = model(x)
            loss = F.mse_loss(x_bar, x)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print("epoch {} loss={:.4f}".format(epoch,
                                            total_loss / (batch_idx + 1)))
        torch.save(model.state_dict(), args.pretrain_path)
    print("model saved to {}.".format(args.pretrain_path))


def train_idec(model,pretrain_epochs=200,train_epochs=100):

#     model = IDEC(
#         n_enc_1=500,
#         n_enc_2=500,
#         n_enc_3=1000,
#         n_dec_1=1000,
#         n_dec_2=500,
#         n_dec_3=500,
#         n_input=args.n_input,
#         n_z=args.n_z,
#         n_clusters=args.n_clusters,
#         alpha=1.0,
#         pretrain_path=args.pretrain_path).to(device)

    #  model.pretrain('data/ae_mnist.pkl')
    model.pretrain(pretrain_epochs=pretrain_epochs)

    train_loader = DataLoader(
        dataset, batch_size=args.batch_size, shuffle=False)
    optimizer = Adam(model.parameters(), lr=args.lr)

    # cluster parameter initiate
    data = dataset.x
    y = dataset.y
    data = torch.Tensor(data).to(device)
    x_bar, hidden = model.ae(data)

    kmeans = KMeans(n_clusters=args.n_clusters, n_init=20)
    y_pred = kmeans.fit_predict(hidden.data.cpu().numpy())
    nmi_k = nmi_score(y_pred, y)
    print("nmi score={:.4f}".format(nmi_k))

    hidden = None
    x_bar = None

    y_pred_last = y_pred
    model.cluster_layer.data = torch.tensor(kmeans.cluster_centers_).to(device)

    model.train()
    for epoch in range(train_epochs):

        if epoch % args.update_interval == 0:

            _, tmp_q = model(data)

            # update target distribution p
            tmp_q = tmp_q.data
            p = target_distribution(tmp_q)

            # evaluate clustering performance
            y_pred = tmp_q.cpu().numpy().argmax(1)
            delta_label = np.sum(y_pred != y_pred_last).astype(
                np.float32) / y_pred.shape[0]
            y_pred_last = y_pred

            acc = cluster_acc(y, y_pred)
            nmi = nmi_score(y, y_pred)
            ari = ari_score(y, y_pred)
            print('Iter {}'.format(epoch), ':Acc {:.4f}'.format(acc),
                  ', nmi {:.4f}'.format(nmi), ', ari {:.4f}'.format(ari))

            if epoch > 0 and delta_label < args.tol:
                print('delta_label {:.4f}'.format(delta_label), '< tol',
                      args.tol)
                print('Reached tolerance threshold. Stopping training.')
                break
        for batch_idx, (x, _, idx) in enumerate(train_loader):

            x = x.to(device)
            idx = idx.to(device)

            x_bar, q = model(x)

            reconstr_loss = F.mse_loss(x_bar, x)
            kl_loss = F.kl_div(q.log(), p[idx])
            loss = args.gamma * kl_loss + reconstr_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [ ]:
data_dir = '/athena/listonlab/store/amb2022/PCMF'

from types import SimpleNamespace
args = SimpleNamespace(
)
batch_size_options = [15, 30]
pretrain_epochs_options = [100, 1000]
train_epochs_options = [100, 1000]

accuracies = []
idx = 0
for batch_size in batch_size_options:
    for pretrain_epochs in pretrain_epochs_options:
        for train_epochs in train_epochs_options:
            args.n_z = 10
            args.lr = 0.001
            args.n_clusters = 4
            args.dataset = 'Synthetic'
            args.gamma = 0.1
            args.update_interval = 1
            args.tol = 0.001
            args.batch_size = batch_size

            os.chdir(data_dir)
            if args.dataset == 'Synthetic':
                args.pretrain_path = 'data/ae_Synthetic.pkl'
                dataset = SyntheticDataset()
                args.n_input = dataset.x.shape[1]
                args.n_clusters = len(np.unique(dataset.y))
            device = 'cpu'
            print(args)

            tic = time.time()
            Synthetic_model = IDEC(
                    n_enc_1=500,
                    n_enc_2=500,
                    n_enc_3=1000,
                    n_dec_1=1000,
                    n_dec_2=500,
                    n_dec_3=500,
                    n_input=args.n_input,
                    n_z=args.n_z,
                    n_clusters=args.n_clusters,
                    alpha=1.0,
                    pretrain_path=args.pretrain_path).to(device)

            train_idec(Synthetic_model,pretrain_epochs=pretrain_epochs,train_epochs=train_epochs)

            toc = time.time() - tic
            print('Time elapsed:',toc)

            data = dataset.x
            y = dataset.y
            data = torch.Tensor(data).to(device)
            x_bar, hidden = Synthetic_model.ae(data)

            # evaluate clustering performance
            _, tmp_q = Synthetic_model(data)

            # update target distribution p
            tmp_q = tmp_q.data
            p = target_distribution(tmp_q)

            y_pred = tmp_q.cpu().numpy().argmax(1)

            acc = cluster_acc(y, y_pred)
            nmi = nmi_score(y, y_pred)
            ari = ari_score(y, y_pred)
            print('Acc {:.4f}'.format(acc),
                  ', nmi {:.4f}'.format(nmi), ', ari {:.4f}'.format(ari))
            
            # Calculate accuracy
            conf_mat_ord = confusion_matrix_ordered(y_pred, y)
            acc = np.sum(np.diag(conf_mat_ord))/np.sum(conf_mat_ord)
            print('IDX:',idx, 'Accuracy:', acc, 'Batch size:',batch_size, 'pretrain_epochs:',pretrain_epochs, 'train_epochs:',train_epochs)
            idx = idx+1

            accuracies.append([idx,acc, batch_size, pretrain_epochs, pretrain_epochs])

        

(100000,)
samples (100000, 10) (100000,)
namespace(batch_size=15, dataset='Synthetic', gamma=0.1, lr=0.001, n_clusters=4, n_input=10, n_z=10, pretrain_path='data/ae_Synthetic.pkl', tol=0.001, update_interval=1)
AE(
  (enc_1): Linear(in_features=10, out_features=500, bias=True)
  (enc_2): Linear(in_features=500, out_features=500, bias=True)
  (enc_3): Linear(in_features=500, out_features=1000, bias=True)
  (z_layer): Linear(in_features=1000, out_features=10, bias=True)
  (dec_1): Linear(in_features=10, out_features=1000, bias=True)
  (dec_2): Linear(in_features=1000, out_features=500, bias=True)
  (dec_3): Linear(in_features=500, out_features=500, bias=True)
  (x_bar_layer): Linear(in_features=500, out_features=10, bias=True)
)
epoch 0 loss=0.0117
epoch 1 loss=0.0065
epoch 2 loss=0.0059
epoch 3 loss=0.0054
epoch 4 loss=0.0052
epoch 5 loss=0.0046
epoch 6 loss=0.0042
epoch 7 loss=0.0035
epoch 8 loss=0.0034
epoch 9 loss=0.0030
epoch 10 loss=0.0029
epoch 11 loss=0.0028
epoch 12 loss=0.0028
